In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.10.0'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# 당뇨병을 진단해봅시다.

In [2]:
xy = np.loadtxt('data-03-diabetes.csv',
                delimiter=',',
                dtype=np.float32)
x_train = xy[0:-100, 0:-1]
y_train = xy[0:-100, [-1]]

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(659, 8) (659, 1)
(100, 8) (100, 1)


In [3]:
print(x_test[0])
print(y_test[0])

[-0.294118  -0.0150754 -0.0491803 -0.333333  -0.550827   0.0134128
 -0.699402  -0.266667 ]
[1.]


In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


Metal device set to: Apple M1


2023-12-19 16:59:36.300745: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 16:59:36.301656: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
W = tf.Variable(tf.random.normal([1,8],1,1), name='weight')
b = tf.Variable(tf.random.normal([1],1,1), name='bias')
tf.print(W,b)

[[0.254617214 0.183739066 0.176341116 ... 0.936546445 1.6796267 0.132879734]] [-0.354415894]


## 가설 설정
* 병이 있다 / 없다로 분류
* sigmoid classification으로 진행


## $$ y_k = \frac{1}{1+e^{-x}}   $$

In [7]:
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(-(tf.matmul(features, tf.transpose(W)) + b)))
    return hypothesis



print(logistic_regression(x_train))

tf.Tensor(
[[0.18095122]
 [0.06941205]
 [0.19711345]
 [0.04901598]
 [0.7179163 ]
 [0.09891399]
 [0.06313179]
 [0.1268881 ]
 [0.11538229]
 [0.15888163]
 [0.13265184]
 [0.24643   ]
 [0.48508045]
 [0.10822792]
 [0.15723322]
 [0.24548307]
 [0.12196535]
 [0.08607066]
 [0.10522775]
 [0.20324388]
 [0.18368296]
 [0.24016896]
 [0.10029022]
 [0.07923076]
 [0.17317645]
 [0.05167257]
 [0.07166258]
 [0.15608056]
 [0.14203118]
 [0.1909814 ]
 [0.03641776]
 [0.08640847]
 [0.13470572]
 [0.15858206]
 [0.19235374]
 [0.19249624]
 [0.14784154]
 [0.4397644 ]
 [0.07542247]
 [0.2807852 ]
 [0.06095279]
 [0.25287196]
 [0.13342525]
 [0.65271336]
 [0.1551046 ]
 [0.09227946]
 [0.12127394]
 [0.1343755 ]
 [0.04462801]
 [0.05295433]
 [0.05415964]
 [0.16766384]
 [0.22297381]
 [0.03820652]
 [0.13735065]
 [0.42688414]
 [0.6722194 ]
 [0.11255766]
 [0.12639573]
 [0.14788826]
 [0.14021456]
 [0.1140916 ]
 [0.13700859]
 [0.06848935]
 [0.21025258]
 [0.32022616]
 [0.03561987]
 [0.06410606]
 [0.12523033]
 [0.09659936]
 [0.30852

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [8]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + \
                           (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.001)

## 학습

In [11]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
        hypothesis = logistic_regression(features)
        loss_value = loss_fn(hypothesis,labels)
    grads = tape.gradient(loss_value, [W,b])
    optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
    if step % 100 == 0:
        print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

Iter: 0, Loss: 1.5734
Iter: 100, Loss: 1.5099
Iter: 200, Loss: 1.4492
Iter: 300, Loss: 1.3916
Iter: 400, Loss: 1.3370
Iter: 500, Loss: 1.2855
Iter: 600, Loss: 1.2371
Iter: 700, Loss: 1.1918
Iter: 800, Loss: 1.1495
Iter: 900, Loss: 1.1102
Iter: 1000, Loss: 1.0738
Iter: 1100, Loss: 1.0402
Iter: 1200, Loss: 1.0092
Iter: 1300, Loss: 0.9808
Iter: 1400, Loss: 0.9547
Iter: 1500, Loss: 0.9309
Iter: 1600, Loss: 0.9092
Iter: 1700, Loss: 0.8894
Iter: 1800, Loss: 0.8713
Iter: 1900, Loss: 0.8549
Iter: 2000, Loss: 0.8400
Iter: 2100, Loss: 0.8264
Iter: 2200, Loss: 0.8141
Iter: 2300, Loss: 0.8029
Iter: 2400, Loss: 0.7927
Iter: 2500, Loss: 0.7835
Iter: 2600, Loss: 0.7750
Iter: 2700, Loss: 0.7673
Iter: 2800, Loss: 0.7603
Iter: 2900, Loss: 0.7539
Iter: 3000, Loss: 0.7481
Iter: 3100, Loss: 0.7427
Iter: 3200, Loss: 0.7378
Iter: 3300, Loss: 0.7332
Iter: 3400, Loss: 0.7291
Iter: 3500, Loss: 0.7252
Iter: 3600, Loss: 0.7216
Iter: 3700, Loss: 0.7183
Iter: 3800, Loss: 0.7153
Iter: 3900, Loss: 0.7124
Iter: 4000, 

## 테스트

In [12]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [13]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.6100
